本章小结：

- 项目对张量使用**广义**爱因斯坦求和约定
- 代码层面，总和可读性与效率，最优的选择是numpy(>1.12)的 np.einsum('ij,jk,kl->il', A, B, C, optimize=True)函数
- 最高效的是np.dot，因为调用的是BLAS库，但是所需的额外优化代码量很大，而且不是那么直观
- 最低效的是不带optimize的np.einsum

# Tensor Manipulation: Psi4 and NumPy manipulation routines

爱因斯坦求和约定，用于计算内积：

$$c = \sum_{ij} A_{ij} * B_{ij}$$

可以简写成：

$$c = A_{ij} * B_{ij}$$

另外，用于矩阵时：

\begin{align}
\rm{Conventional}\;\;\;  C_{ik} &= \sum_{j} A_{ij} * B_{jk} \\
\rm{Einstein}\;\;\;  C &= A_{ij} * B_{jk} \\
\end{align}

然而，有些情况下，这个求和约定无法使用，此时需要适当扩展记号，比如对于矩阵的星乘（元素相乘）：

$$C_{ij} = \sum_{ij} A_{ij} * B_{ij}$$

左边就必须加上下标，否则无法区分是点乘还是星乘。

$$C_{ij} = A_{ij} * B_{ij}$$

对于矩阵：

\begin{align}
{\rm Matrix}\;\;\;  \bf{D} &= \bf{A B C} \\
{\rm Einstein}\;\;\;  D_{il} &= A_{ij} B_{jk} C_{kl}
\end{align}

In [3]:
import numpy as np

A = np.random.random((2,3))
B = np.random.random((2,3))
c = np.inner(A,B)
print(c)

C = A*B
print(C)

[[1.03932478 1.372394  ]
 [0.81406273 1.20047283]]
[[0.7394282  0.21402241 0.08587418]
 [0.28354254 0.80258753 0.11434275]]


## Einsum

To perform most operations we turn to [NumPy's einsum function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html) which allows the Einsten convention as an input. In addition to being much easier to read, manipulate, and change, it is also much more efficient that a pure Python implementation.

To begin let us consider the construction of the following tensor (which you may recognize):
$$G_{pq} = 2.0 * I_{pqrs} D_{rs} - 1.0 * I_{prqs} D_{rs}$$ 

First let us import our normal suite of modules:

In [1]:
import numpy as np
import psi4
import time

We can then use conventional Python loops and einsum to perform the same task. Keep size relatively small as these 4-index tensors grow very quickly in size.

In [2]:
size = 20

if size > 30:
    raise Exception("Size must be smaller than 30.")
D = np.random.rand(size, size)
I = np.random.rand(size, size, size, size)

# Build the fock matrix using loops, while keeping track of time
tstart_loop = time.time()
Gloop = np.zeros((size, size))
for p in range(size):
    for q in range(size):
        for r in range(size):
            for s in range(size):
                Gloop[p, q] += 2 * I[p, q, r, s] * D[r, s]
                Gloop[p, q] -=     I[p, r, q, s] * D[r, s]

g_loop_time = time.time() - tstart_loop

# Build the fock matrix using einsum, while keeping track of time
tstart_einsum = time.time()
J = np.einsum('pqrs,rs', I, D, optimize=True)
K = np.einsum('prqs,rs', I, D, optimize=True)
G = 2 * J - K

einsum_time = time.time() - tstart_einsum

# Make sure the correct answer is obtained
print('The loop and einsum fock builds match:    %s\n' % np.allclose(G, Gloop))
# Print out relative times for explicit loop vs einsum Fock builds
print('Time for loop G build:   %14.4f seconds' % g_loop_time)
print('Time for einsum G build: %14.4f seconds' % einsum_time)
print('G builds with einsum are {:3.4f} times faster than Python loops!'.format(g_loop_time / einsum_time))

The loop and einsum fock builds match:    True

Time for loop G build:           0.4252 seconds
Time for einsum G build:         0.0277 seconds
G builds with einsum are 15.3570 times faster than Python loops!


As you can see, the einsum function is considerably faster than the pure Python loops and, in this author's opinion, much cleaner and easier to use.

## Dot

Now let us turn our attention to a more canonical matrix multiplication example such as:
$$D_{il} = A_{ij} B_{jk} C_{kl}$$

We could perform this operation using einsum; however, matrix multiplication is an extremely common operation in all branches of linear algebra. Thus, these functions have been optimized to be more efficient than the `einsum` function. The matrix product will explicitly compute the following operation:
$$C_{ij} = A_{ij} * B_{ij}$$

This can be called with [NumPy's dot function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html#numpy.dot).

In [3]:
size = 200
A = np.random.rand(size, size)
B = np.random.rand(size, size)
C = np.random.rand(size, size)

# First compute the pair product
tmp_dot = np.dot(A, B)
tmp_einsum = np.einsum('ij,jk->ik', A, B, optimize=True)
print("Pair product allclose: %s" % np.allclose(tmp_dot, tmp_einsum))

Pair product allclose: True


Now that we have proved exactly what the dot product does, let us consider the full chain and do a timing comparison:

In [4]:
D_dot = np.dot(A, B).dot(C)
D_einsum = np.einsum('ij,jk,kl->il', A, B, C, optimize=True)
print("Chain multiplication allclose: %s" % np.allclose(D_dot, D_einsum))

print("\nnp.dot time:")
%timeit np.dot(A, B).dot(C)

print("\nnp.einsum time")
# no optimization here for illustrative purposes!
%timeit np.einsum('ij,jk,kl->il', A, B, C)

Chain multiplication allclose: True

np.dot time:
1.25 ms ± 255 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

np.einsum time
1.89 s ± 56.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


On most machines the `np.dot` times are roughly ~2,000 times faster. The reason is twofold:
 - The `np.dot` routines typically call [Basic Linear Algebra Subprograms (BLAS)](https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms). The BLAS routines are highly optimized and threaded versions of the code.
 - The `np.einsum` code will not factorize the operation by default; Thus, the overall cost is ${\cal O}(N^4)$ (as there are four indices) rather than the factored $(\bf{A B}) \bf{C}$ which runs ${\cal O}(N^3)$.
 
The first issue is difficult to overcome; however, the second issue can be resolved by the following:

In [5]:
print("np.einsum factorized time:")
# no optimization here for illustrative purposes!
%timeit np.einsum('ik,kl->il', np.einsum('ij,jk->ik', A, B), C)

np.einsum factorized time:
6.93 ms ± 294 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


On most machines the factorized `einsum` expression is only ~10 times slower than `np.dot`. While a massive improvement, this is a clear demonstration the BLAS usage is usually recommended. It is a tradeoff between speed and readability. The Psi4NumPy project tends to lean toward `einsum` usage except in case where the benefit is too large to pass up.

Starting in NumPy 1.12, the [einsum function](https://docs.scipy.org/doc/numpy/reference/generated/numpy.einsum.html) has a `optimize` flag which will automatically factorize the einsum code for you using a greedy algorithm, leading to considerable speedups at almost no cost:

In [6]:
print("\nnp.einsum optimized time")
%timeit np.einsum('ij,jk,kl->il', A, B, C, optimize=True)


np.einsum optimized time
1.57 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In this example, using `optimize=True` for automatic factorization is only 25% slower than `np.dot`. Furthermore, it is ~5 times faster than factorizing the expression by hand, which represents a very good trade-off between speed and readability. When unsure, `optimize=True` is strongly recommended.

## Complex tensor manipulations

用一个更复杂的例子展示这些方法之间的效率差别：

Let us consider a popular index transformation example:
$$M_{pqrs} = C_{pi} C_{qj} I_{ijkl} C_{rk} C_{sl}$$

Here, a naive `einsum` call would scale like $\mathcal{O}(N^8)$ which translates to an extremely costly computation for all but the smallest $N$.

In [7]:
# Grab orbitals
size = 15
if size > 15:
    raise Exception("Size must be smaller than 15.")
    
C = np.random.rand(size, size)
I = np.random.rand(size, size, size, size)

# Numpy einsum N^8 transformation.
print("\nStarting Numpy's N^8 transformation...")
n8_tstart = time.time()
# no optimization here for illustrative purposes!
MO_n8 = np.einsum('pI,qJ,pqrs,rK,sL->IJKL', C, C, I, C, C)
n8_time = time.time() - n8_tstart
print("...transformation complete in %.3f seconds." % (n8_time))

# Numpy einsum N^5 transformation.
print("\n\nStarting Numpy's N^5 transformation with einsum...")
n5_tstart = time.time()
# no optimization here for illustrative purposes!
MO_n5 = np.einsum('pA,pqrs->Aqrs', C, I)
MO_n5 = np.einsum('qB,Aqrs->ABrs', C, MO_n5)
MO_n5 = np.einsum('rC,ABrs->ABCs', C, MO_n5)
MO_n5 = np.einsum('sD,ABCs->ABCD', C, MO_n5)
n5_time = time.time() - n5_tstart
print("...transformation complete in %.3f seconds." % n5_time)
print("\nN^5 %4.2f faster than N^8 algorithm!" % (n8_time / n5_time))
print("Allclose: %s" % np.allclose(MO_n8, MO_n5))

# Numpy einsum optimized transformation.
print("\nNow Numpy's optimized transformation...")
n8_tstart = time.time()
MO_n8 = np.einsum('pI,qJ,pqrs,rK,sL->IJKL', C, C, I, C, C, optimize=True)
n8_time_opt = time.time() - n8_tstart
print("...optimized transformation complete in %.3f seconds." % (n8_time_opt))

# Numpy GEMM N^5 transformation.
# Try to figure this one out!
print("\n\nStarting Numpy's N^5 transformation with dot...")
dgemm_tstart = time.time()
MO = np.dot(C.T, I.reshape(size, -1))
MO = np.dot(MO.reshape(-1, size), C)
MO = MO.reshape(size, size, size, size).transpose(1, 0, 3, 2)

MO = np.dot(C.T, MO.reshape(size, -1))
MO = np.dot(MO.reshape(-1, size), C)
MO = MO.reshape(size, size, size, size).transpose(1, 0, 3, 2)
dgemm_time = time.time() - dgemm_tstart
print("...transformation complete in %.3f seconds." % dgemm_time)
print("\nAllclose: %s" % np.allclose(MO_n8, MO))
print("N^5 %4.2f faster than N^8 algorithm!" % (n8_time / dgemm_time))


Starting Numpy's N^8 transformation...
...transformation complete in 26.988 seconds.


Starting Numpy's N^5 transformation with einsum...
...transformation complete in 0.004 seconds.

N^5 6144.96 faster than N^8 algorithm!
Allclose: True

Now Numpy's optimized transformation...
...optimized transformation complete in 0.002 seconds.


Starting Numpy's N^5 transformation with dot...
...transformation complete in 0.001 seconds.

Allclose: True
N^5 18744.20 faster than N^8 algorithm!
